In [ ]:
f_schedules_oag = '../data/oag_schedules/OAG_global_6SEP2019.csv'
f_airports = '../data/airports/airports_coordinates_updated.csv'
f_airports_iata_icao = '../data/airports/IATA_ICAO_Airport_codes_v0.3.csv'

countries_interest = ['ES'] #, 'DE']
#airports_intesrest = ['LHR', 'CDG', 'FRA', 'MUC', 'AMS', 'LIS', 'FCO', 'ZRH']
airports_intesrest = []
general_ac_remove = ['TRN', 'BUS', 'LCH', 'AWH'] # Trains, Bus, Launch (boat), AWH (helicopter)
specific_ac_remove = ['RFS'] # RFS - Road Feeder Service (Truck)

f_schedules_output = '../data/flight_schedules_oag_es.csv'

In [ ]:
import pandas as pd
df_schedules_oag = pd.read_csv(f_schedules_oag)
df_airports = pd.read_csv(f_airports)
df_airports_iata_icao = pd.read_csv(f_airports_iata_icao)

In [ ]:
len(df_schedules_oag[df_schedules_oag.DepAirport=='MAD'])

In [ ]:
# Checking that airport icao-iata don't have duplicate IATA code, as otherwise when doing the merge the flights get duplicated...
df_airports_iata_icao[['IATA']].drop_duplicates()
df_airports_iata_icao[df_airports_iata_icao.duplicated(subset=['IATA'], keep=False)]['IATA'].unique()

In [ ]:
# Remove codeshares because airline share is not null (i.e., another airline is doing the flight)
# The comment above was the idea I had, but it seems that two flights can have both ShAirlDes and one of them
# be operated, (see example of MAD-LIS flights). So dont' do this. Instead see code later for codeshares
# There can still be codeshares, see later
print(len(df_schedules_oag[df_schedules_oag.DepAirport=='MAD']))
print(len(df_schedules_oag[df_schedules_oag.DepAirport=='BCN']))
#df_schedules_oag = df_schedules_oag[df_schedules_oag.ShAirlDes.isnull()].copy()
print(len(df_schedules_oag[df_schedules_oag.DepAirport=='MAD']))
print(len(df_schedules_oag[df_schedules_oag.DepAirport=='BCN']))

In [ ]:
# vdf_schedules_oag[((df_schedules_oag.Carrier1=='QR') | (df_schedules_oag.Carrier1=='VY') | (df_schedules_oag.Carrier1=='IB')) & (df_schedules_oag.DepAirport=='PMI') & ((df_schedules_oag.ArrAirport=='BCN'))].to_csv('../data/oag_schedules/test_code_share.csv')

In [ ]:
# df_schedules_oag_grouped = df_schedules_oag.groupby(['DepAirport', 'ArrAirport', 'LocalDepTime', 'LocalArrTime'])
# filtered_df = df_schedules_oag_grouped.filter(lambda x: len(x) > 1)
# filtered_df.to_csv('../data/oag_schedules/same_de_arr_times.csv')

In [ ]:
# Create unique service ids for the flights
def make_unique_service_ids(df, column):
    seen = {}
    unique_ids = []
    
    for service_id in df[column]:
        if service_id in seen:
            seen[service_id] += 1
            unique_id = f"{service_id}_{seen[service_id]}"
        else:
            seen[service_id] = 0
            unique_id = service_id
        
        unique_ids.append(unique_id)
    
    return unique_ids


df_schedules_oag['service_id'] = df_schedules_oag['Carrier1'] + '_' + df_schedules_oag['FlightNo1'].astype(str)

df_schedules_oag['service_id'] = make_unique_service_ids(df_schedules_oag, 'service_id')


In [ ]:
print(len(df_schedules_oag[df_schedules_oag['DepAirport']=='MAD']))
print(len(df_schedules_oag[df_schedules_oag['DepAirport']=='BCN']))

In [ ]:
# Filter operations that we could be interested

# Filter data to keep one of interest
print(len(df_schedules_oag))
df_s = df_schedules_oag[df_schedules_oag.ArrIATACtry.isin(countries_interest) | 
                        df_schedules_oag.DepIATACtry.isin(countries_interest) |
                        df_schedules_oag.DepAirport.isin(airports_intesrest) |
                        df_schedules_oag.ArrAirport.isin(airports_intesrest)].copy()
print(len(df_s))
print(len(df_s[df_s['DepAirport']=='MAD']))
print(len(df_s[df_s['DepAirport']=='BCN']))


In [ ]:
print(len(df_s))
# Further filter things that are not flights or we don't care
df_s = df_s[~df_s.SpecificAcft.isin(specific_ac_remove) &
            ~df_s.GeneralAcft.isin(general_ac_remove)]
print(len(df_s))

# Remove circular flights
df_s = df_s[df_s.DepAirport!=df_s.ArrAirport]
print(len(df_s))


df_s = df_s.copy().reset_index(drop=True)


In [ ]:
# Add ICAO origin and destination
df_s = df_s.merge(df_airports_iata_icao[['ICAO', 'IATA']], left_on='DepAirport', right_on='IATA', how='left').rename(columns={'ICAO': 'origin'}).drop(columns={'IATA'})
df_s = df_s.merge(df_airports_iata_icao[['ICAO', 'IATA']], left_on='ArrAirport', right_on='IATA', how='left').rename(columns={'ICAO': 'destination'}).drop(columns={'IATA'})

In [ ]:
# Identify ariprots missing their ICAO code in the IATA-ICAO file
print(df_s[df_s.origin.isnull()][['DepAirport', 'DepAirportName']].drop_duplicates())
print(df_s[df_s.destination.isnull()][['ArrAirport', 'ArrAirportName']].drop_duplicates())

In [ ]:
# Get coordinates of airports
# Merge with df_s to add lat lon of arrival and departure airports
df_s = df_s.merge(df_airports[['icao_id', 'lat', 'lon']], left_on='origin', right_on='icao_id', how='left').drop(columns={'icao_id'}).rename(columns={'lat':'DepLat', 'lon':'DepLon'})
df_s = df_s.merge(df_airports[['icao_id', 'lat', 'lon']], left_on='destination', right_on='icao_id', how='left').drop(columns={'icao_id'}).rename(columns={'lat':'ArrLat', 'lon':'ArrLon'})


In [ ]:
# Coordinates missing
print(df_s[df_s.DepLat.isnull()]['origin'].drop_duplicates())
print(df_s[df_s.ArrLat.isnull()]['destination'].drop_duplicates())

In [ ]:
first_columns = ['Carrier1', 'Carrier1Name', 'FlightNo1', 'CarrDom1', 'CarrDom1Name', 'DepAirport', 'DepAirportName', 'DepLat', 'DepLon', 'DepTerminal', 'DepCity', 'DepCityName', 'DepState', 'DepStateName', 'DepIATACtry', 'DepIATACtryName', 'DepDOTCtry', 'DepDOTCtryName', 'DepReg', 'DepRegName', 'ArrAirport', 'ArrAirportName', 'ArrLat', 'ArrLon', 'ArrTerminal', 'ArrCity', 'ArrCityName', 'ArrState', 'ArrStateName', 'ArrIATACtry', 'ArrIATACtryName', 'ArrDOTCtry', 'ArrDOTCtryName', 'ArrReg', 'ArrRegName', 'LocalDepTime', 'LocalArrTime', 'LocalArrDay', 'LocalDaysOfOp', 'ArrDaysOfOp', 'Service', 'Seats', 'FstSeats', 'BusSeats', 'EcoSeats', 'EffFrom', 'EffTo', 'LocalDaysOfOp1', 'LocalDaysOfOp2', 'LocalDaysOfOp3', 'LocalDaysOfOp4', 'LocalDaysOfOp5', 'LocalDaysOfOp6', 'LocalDaysOfOp7', 'ElapsedTime', 'FlyingTime', 'GroundTime', 'Stops']
mid_columns = ['IntAirports', 'IntCities', 'IntCountries', 'AcftChange', 'AcftChApt1', 'AcftChApt2', 'AcftChApt3', 'GeneralAcft', 'GeneralAcftName', 'SpecificAcft', 'SpecificAcftName', 'SecondAcft', 'ThirdAcft', 'FourthAcft', 'Freightons', 'PassClass', 'FreightClass', 'Routing', 'StatMiles', 'NautMiles', 'Km', 'DistStMiles', 'DistNtMiles', 'DistKM', 'Restrictions', 'ShAirlDes', 'ShrAirlineDesName', 'MultCDes', 'DupMarker', 'DupCar1', 'DupCar2', 'DupCar3', 'DupCar4', 'DupCar5', 'DupCar6', 'DupCar7', 'DupCar8', 'OpCar', 'Comment', 'AcftOwnerCode', 'AcftOwnerCodeName', 'CockpitCrewCode', 'CockpitCrewCodeName', 'CabinCrewCode', 'CabinCrewCodeName', 'LongLeg', 'MaxTakeOffWeight', 'HoldVolume', 'RangeKm', 'RangeStatMiles', 'RangeNautMiles', 'CruiseSpeed', 'Category', 'Manufacturer', 'Ghost']
final_columns = ['SubGovnApp', 'FltDup', 'Frequency', 'ASMs', 'ASKs', 'TotalSeatCapacity', 'TotalTonnage']

In [ ]:
# df_s[first_columns].loc[4]

In [ ]:
# df_s_es[mid_columns].loc[4]

In [ ]:
# df_s_es[final_columns].loc[4]

In [ ]:
# Move dates to UTC from local
from datetime import datetime, timedelta
import pytz
from timezonefinder import TimezoneFinder

# Initialize TimezoneFinder
tf = TimezoneFinder()

# Function to convert local times to UTC
def convert_to_utc(row, reference_date):
    # Define the departure time zones
    dep_airport = row['DepAirport']
    dep_state = row['DepState']

    # Adjust reference date if arrival is next day and not in Europe
    # Move date to -1 so that when it arrives to Europe it is in the
    # reference date
    dep_date = datetime.strptime(reference_date, '%Y-%m-%d')
    if row['LocalArrDay'] == 1 and row['DepReg'] != 'EU1' and row['DepReg'] != 'EU2':
        dep_date -= timedelta(days=1)

    # Get the timezone of the departure airport, either by its coordinates
    # or if coordinates not available then by the country of the airport
    # note that country might not be good enough (e.g. ES has Canary Islands in a different 
    # time zone or the US has many different timezones)
    if not pd.isna(row['DepLat']):
        dep_tz_str = tf.timezone_at(lng=row['DepLon'], lat=row['DepLat'])
    else:
        # From all the time zones in the country, keep the first one
        dep_tz_str = pytz.country_timezones[row['DepIATACtry']][0]
    
    dep_tz = pytz.timezone(dep_tz_str)
        
    
    # Parse the local departure time
    local_dep_time_str = f"{dep_date.strftime('%Y-%m-%d')} {str(row['LocalDepTime']).zfill(4)[:2]}:{str(row['LocalDepTime']).zfill(4)[2:]}:00"
    local_dep_time = datetime.strptime(local_dep_time_str, '%Y-%m-%d %H:%M:%S')    
    local_dep_time = dep_tz.localize(local_dep_time)
    
    # Convert local departure time to UTC
    utc_dep_time = local_dep_time.astimezone(pytz.utc)
    
    # Calculate the arrival time based on elapsed time
    elapsed_time_str = str(row['ElapsedTime']).zfill(4)
    elapsed_hours = int(elapsed_time_str[:2])
    elapsed_minutes = int(elapsed_time_str[2:])
    elapsed_time = timedelta(hours=elapsed_hours, minutes=elapsed_minutes)

    local_arr_time = local_dep_time + elapsed_time

    # Get the timezone of the arrival airport, either by its coordinates
    if not pd.isna(row['ArrLat']):
        arr_tz_str = tf.timezone_at(lng=row['ArrLon'], lat=row['ArrLat'])
    else:
        arr_tz_str = pytz.country_timezones[row['ArrIATACtry']][0]

    arr_tz = pytz.timezone(arr_tz_str)

    # Convert local arrival time to the correct local timezone
    local_arr_time = local_arr_time.astimezone(arr_tz)
    
    # Convert local arrival time to UTC
    utc_arr_time = local_arr_time.astimezone(pytz.utc)
    
    
    # Convert local arrival time to UTC
    #utc_arr_time = local_arr_time.astimezone(pytz.utc)
    
    return pd.Series([utc_dep_time, utc_arr_time, local_dep_time, local_arr_time])



reference_utc_date = '2019-09-06'
# Apply the function to the DataFrame
df_s[['UTC_Departure', 'UTC_Arrival', 'Local_Departure', 'Local_Arrival']] = df_s.apply(convert_to_utc, reference_date=reference_utc_date, axis=1)





In [ ]:
# Filter flights operating on day of interest

# Function to extract day of the week
def get_day_of_week(local_departure):
    return local_departure.weekday() + 1  # Python's weekday() returns 0 for Monday, so add 1 to match required format

# Function to check if flight operates on the day of the week
def operates_on_day(row):
    return str(row['DayOfWeek']) in row['LocalDaysOfOp']

# Function to check if departure date is within the effective date range
def within_effective_date_range(row):
    dep_date = row['Local_Departure'].date()
    eff_from_date = row['EffFrom'].date()
    eff_to_date = row['EffTo'].date()
    return eff_from_date <= dep_date <= eff_to_date

# Add the DayOfWeek column
df_s['DayOfWeek'] = df_s['Local_Departure'].apply(get_day_of_week)

# Convert EffFrom and EffTo to datetime
df_s['EffFrom'] = pd.to_datetime(df_s['EffFrom'], format='%d/%m/%Y')
df_s['EffTo'] = pd.to_datetime(df_s['EffTo'], format='%d/%m/%Y')

# Filter the DataFrame
df_fs_filtered = df_s[df_s.apply(operates_on_day, axis=1)]
df_fs_filtered = df_fs_filtered[df_fs_filtered.apply(within_effective_date_range, axis=1)]


In [ ]:
# Remove flights with stops
# In original data you can have A - B - C that will appear as three flights:
# - A - B - C (With Stops==1)
# - A - B
# - B - C
# Remove the one with stops and keep the 'simple' ones.
df_fs_filtered = df_fs_filtered[df_fs_filtered.Stops==0].copy()

In [ ]:
print(len(df_fs_filtered[df_fs_filtered['DepAirport']=='MAD']))
print(len(df_fs_filtered[df_fs_filtered['DepAirport']=='BCN']))

In [ ]:
# Now that we have filtered by date of interest, we need to check still codeshares which are not always.
# For exmample between LEPA and LEBL there's a flight by Vueling and Iberia which overlaps on some days, in that case
# the flight is Iberia, on the other days, it's Vueling, for example.

# Function to filter flights
def filter_flights(group):
    # Create the full flight code
    group['FlightCode'] = group['Carrier1'] + ' ' + group['FlightNo1'].astype(str)
    
    # If the group has only one flight, keep it
    if len(group) == 1:
        return group
    else:
        # Print group information before filtering
        #print(f"\nGroup before filtering (DepAirport: {group['DepAirport'].iloc[0]}, ArrAirport: {group['ArrAirport'].iloc[0]}, LocalDepTime: {group['LocalDepTime'].iloc[0]}, LocalArrTime: {group['LocalArrTime'].iloc[0]}):")
        #print(group[['DepAirport', 'ArrAirport', 'LocalDepTime', 'Carrier1', 'FlightNo1', 'FlightCode', 'OpCar']])
        
        # Check if a flight code appears in another flight's DupCarx columns
        codes_to_remove = set()
        flight_codes = set(group['FlightCode'])
        mutual_dups = []

        for _, row in group.iterrows():
            for i in range(1, 9):
                dup_car = row[f'DupCar{i}']
                if dup_car in flight_codes:
                    if dup_car in group['FlightCode'].values and row['FlightCode'] in group.loc[group['FlightCode'] == dup_car, [f'DupCar{j}' for j in range(1, 9)]].values:
                        mutual_dups.append((row['FlightCode'], dup_car))
                    else:
                        codes_to_remove.add(dup_car)

        # Handle mutual duplicates
        for fc1, fc2 in mutual_dups:
            fc1_op = group.loc[group['FlightCode'] == fc1, 'OpCar'].values[0]
            fc2_op = group.loc[group['FlightCode'] == fc2, 'OpCar'].values[0]
            if fc1_op == 'O':
                codes_to_remove.add(fc2)
            elif fc2_op == 'O':
                codes_to_remove.add(fc1)
        
        # Keep flights whose code is not in the codes_to_remove set
        filtered_group = group[~group['FlightCode'].isin(codes_to_remove)]
        
        # Print group information after filtering
        #print(f"\nGroup after filtering (DepAirport: {filtered_group['DepAirport'].iloc[0]}, ArrAirport: {filtered_group['ArrAirport'].iloc[0]}, LocalDepTime: {filtered_group['LocalDepTime'].iloc[0]}, LocalArrTime: {filtered_group['LocalArrTime'].iloc[0]}):")
        #print(filtered_group[['DepAirport', 'ArrAirport', 'LocalDepTime', 'Carrier1', 'FlightNo1', 'FlightCode']])
        #print("----")
        
        # Keep flights whose code is not in the codes_to_remove set
        return filtered_group


# Group by the four columns
grouped = df_fs_filtered.groupby(['DepAirport', 'ArrAirport', 'LocalDepTime', 'LocalArrTime'])


print(len(df_fs_filtered))

# Apply the filter function to each group
df_filtered = grouped.apply(filter_flights).reset_index(drop=True)

print(len(df_filtered))


In [ ]:
print(len(df_fs_filtered[df_fs_filtered['DepAirport']=='MAD']))
print(len(df_fs_filtered[df_fs_filtered['DepAirport']=='BCN']))

In [ ]:
# Print duplicates flights still remaining. It could be legit (two airlines with same o-d at same times...)
# Group by the four columns
grouped = df_filtered.groupby(['DepAirport', 'ArrAirport', 'LocalDepTime', 'LocalArrTime'])

# Filter groups with more than one entry
post_filtered_df = grouped.filter(lambda x: len(x) > 1)

# Print the groups with more than one flight
for name, group in post_filtered_df.groupby(['DepAirport', 'ArrAirport', 'LocalDepTime', 'LocalArrTime']):
    print(f"DepAirport: {name[0]}, ArrAirport: {name[1]}, LocalDepTime: {name[2]}, LocalArrTime: {name[3]}")
    print(group[['DepAirport', 'ArrAirport', 'LocalDepTime', 'Carrier1', 'FlightNo1', 'OpCar', 'DupCar1']])
    print(f"Number of flights in this group: {len(group)}\n")

# Keep the rows that are the only ones in their group
unique_df = grouped.filter(lambda x: len(x) == 1)

# Combine the unique and non-unique groups
#final_df = pd.concat([unique_df, filtered_df])

In [ ]:
# Save the schedules
df_filtered = df_filtered[['service_id', 'origin', 'destination',
                                 'DepTerminal', 'ArrTerminal',
                                 'UTC_Departure', 'UTC_Arrival', 
                                 'Local_Departure', 'Local_Arrival',
                                 'Carrier1', 'SpecificAcft', 'Seats', 'DistKM']].copy()

df_filtered = df_filtered.rename(columns={'DepTerminal': 'dep_terminal',
                               'ArrTerminal': 'arr_terminal',
                               'UTC_Departure':'sobt',
                               'UTC_Arrival':'sibt',
                               'Local_Departure': 'sobt_local',
                               'Local_Arrival': 'sibt_local',
                               'Carrier1':'provider',
                               'SpecificAcft':'act_type',
                               'Seats': 'seats',
                               'DistKM':'gcdistance'})#to_csv('../data/oag_schedules/schedules_test.csv', index=False)#, inplace=True)



# Function to extract the offset in hours and minutes
def extract_offset(x):
    x = str(x)
    if len(x.split("+")) > 1:
        return "+"+x.split("+")[-1]
    else:
        return "-"+x.split("-")[-1]
    
    
# Extract timezone information
df_filtered['sobt_tz'] = df_filtered['sobt'].apply(lambda x: extract_offset(x))
df_filtered['sobt_local_tz'] = df_filtered['sobt_local'].apply(lambda x: extract_offset(x))
df_filtered['sibt_tz'] = df_filtered['sibt'].apply(lambda x: extract_offset(x))
df_filtered['sibt_local_tz'] = df_filtered['sibt_local'].apply(lambda x: extract_offset(x))

# Remove timezone information from datetime columns
df_filtered['sobt'] = df_filtered['sobt'].dt.tz_localize(None)
df_filtered['sobt_local'] = df_filtered['sobt_local'].apply(lambda x: x.replace(tzinfo=None))
df_filtered['sibt'] = df_filtered['sibt'].dt.tz_localize(None)
df_filtered['sibt_local'] = df_filtered['sibt_local'].apply(lambda x: x.replace(tzinfo=None))

# Remove flights which are same airline and all the same but the flight service
df_filtered = df_filtered.drop_duplicates(subset=[col for col in df_filtered.columns if col !='service_id'], keep='first')

print(len(df_filtered))




In [ ]:
# Print again duplicates flights still remaining. It could be legit (two airlines with same o-d at same times...)
# Group by the four columns
grouped = df_filtered.groupby(['origin', 'destination', 'sobt', 'sibt'])

# Filter groups with more than one entry
post_filtered_df = grouped.filter(lambda x: len(x) > 1)

n_dup = 0
# Print the groups with more than one flight
for name, group in post_filtered_df.groupby(['origin', 'destination', 'sobt', 'sibt']):
    n_dup += 1
    print(f"DepAirport: {name[0]}, ArrAirport: {name[1]}, LocalDepTime: {name[2]}, LocalArrTime: {name[3]}")
    print(group[['origin', 'destination', 'sobt', 'provider', 'act_type']])
    print(f"Number of flights in this group: {len(group)}\n")

print(f"Total number of groups: {n_dup}\n")

In [ ]:
# Modify seats of act_type that are missing their seats
if len(df_filtered[df_filtered.seats==0].act_type.drop_duplicates())>0:
    pd_seats_missing = pd.read_csv('./seats_ac_missing.csv')

dict_ac_seats = pd_seats_missing.set_index('ac_type').to_dict()['seats']

df_filtered['seats'] = df_filtered.apply(lambda x: dict_ac_seats.get(x['act_type'], x['seats']), axis=1)

df_filtered[df_filtered.seats==0].act_type.drop_duplicates()

In [ ]:
df_filtered[['service_id', 'origin', 'destination', 'dep_terminal', 'arr_terminal',
    'sobt', 'sibt', 'sobt_tz', 'sibt_tz', 
    'sobt_local', 'sibt_local','sobt_local_tz', 'sibt_local_tz',
    'provider', 'act_type',
    'seats', 'gcdistance']].to_csv(f_schedules_output, index=False)


In [ ]:
print(len(df_filtered))
len(df_filtered.drop_duplicates(subset=[col for col in df_filtered.columns if col !='service_id'], keep='first'))

In [ ]:
df_filtered[((df_filtered.provider=='QR') | (df_filtered.provider=='VY') | (df_filtered.provider=='IB')) & 
(df_filtered.origin=='LEPA') & ((df_filtered.destination=='LEBL'))].sort_values('sobt')

In [ ]:
df_filtered[(df_filtered.origin=='EDDL') & (df_filtered.destination=='LEPA')].sort_values('sobt')